In [188]:
import pandas as pd
import datetime

from elasticsearch import Elasticsearch
from pandas.io.json import json_normalize

def to_date(s):
    dt = datetime.datetime.fromtimestamp(s / 1e3)
    return dt.strftime("%Y-%m-%d")

begin_time='2019-01-07'
end_time = '2019-05-27'
cal_period = '7D'
dsl = {
        'query': {'bool': {'must': [
                                    {'terms': {'order_type': ['vip_scan']}},
                                    {'terms': {'order_status': ['confirmed',
                                                                'done',
                                                                'fulfilled']}},
                                    ]}},
        "aggs": {
            "my_buckets": {
                "composite": {
                    'size': 9999999,
                    "sources": [
                        {"日期": {"date_histogram": {"field": "paytime", "interval": "1d"}}},
                        
                        {"门店": {"terms": {"field": 'shop_id'}}},
                    ]
                }
            }
        }
    }
es = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
res = es.search(index='es_order_table', doc_type='order', body=dsl,
                _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                search_type='query_then_fetch')
df = json_normalize(res['aggregations']['my_buckets']['buckets'])
df.columns = map(lambda x: x.split('key.')[-1] if x.startswith('key.') else x, list(df.columns))
df['日期'] = df['日期'].apply(to_date)
date

'2019-05-27'

In [179]:
df = df[df['日期']>=begin_time]

In [180]:
def get_first_date(s):
    data = {}
    data['first_date'] = s['日期'].min()
    return pd.Series(data)
dts = df.groupby('门店').apply(get_first_date).reset_index()
dts.head()

,门店,first_date
0,358,2019-03-26
1,365,2019-03-25
2,498,2019-04-19
3,501,2019-04-10
4,504,2019-01-31


In [146]:
dts['first_date'] = pd.to_datetime(dts['first_date']) 

In [147]:
def get_week_shop_list(s):
    data = {}
    data['week_new_shop_list'] = s['门店'].unique().tolist()
    return pd.Series(data)
dts = dts.resample('7d', on = 'first_date').apply(get_week_shop_list).reset_index()

In [185]:
dts.head()

In [186]:
date

'2019-05-27'

In [149]:
this_shop_count_list,this_gmv_count_list,this_order_count_list=[],[],[]
for i in range(len(dts)):
    begin_date = str(dts.iloc[i]['first_date'])[:10]
    shop_list = dts.iloc[i]['week_new_shop_list']
    end_date = str(datetime.datetime.strptime(date,'%Y-%m-%d')+datetime.timedelta(days=6))[:10]
    dsl = {'query': {'bool': {'must': [
    {'range': {'paytime': {'gte': begin_date, 'lte': end_date, 'format': 'yyyy-MM-dd', 'time_zone': '+00:00'}}},
    {'terms': {'shop_id': shop_list}}, {'terms': {'order_type': ['vip_scan', 'deal']}},
    {'terms': {'order_status': ['done', 'fulfilled', 'confirmed']}}]}},
     'aggs': {'my_buckets': {'composite': {'size': 9999999, 'sources': [
         {'日期': {'date_histogram': {'field': 'paytime', 'interval': '1w', 'time_zone': '+00:00'}}}]},
                             'aggregations': {'折前GMV': {'sum': {'field': 'skutotal_price'}},
                                              '门店数': {'cardinality': {'field': 'shop_id'}}}}}}
    es = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res = es.search(index='es_order_table', doc_type='order', body=dsl,
                _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                search_type='query_then_fetch')
    if len(res['aggregations']['my_buckets']['buckets'])>0:
        shop_count = res['aggregations']['my_buckets']['buckets'][0]['门店数']['value']
        gmv_count = res['aggregations']['my_buckets']['buckets'][0]['折前GMV']['value']
        order_count = res['aggregations']['my_buckets']['buckets'][0]['doc_count']
    else:
        shop_count = gmv_count = order_count =0
    this_shop_count_list.append(shop_count)
    this_gmv_count_list.append(gmv_count)
    this_order_count_list.append(order_count)
dts['本周门店数'] = this_shop_count_list
dts['本周GMV'] = this_gmv_count_list
dts['本周订单数'] = this_order_count_list

In [170]:

shop_count_list,gmv_count_list,order_count_list=[],[],[]
for i in range(len(dts)):
    date = str(dts.iloc[i]['first_date'])[:10]
    shop_list = dts.iloc[i]['week_new_shop_list']
    interval = 8
    begin_date = str(datetime.datetime.strptime(date,'%Y-%m-%d')+datetime.timedelta(days=7*interval))[:10]
    end_date = str(datetime.datetime.strptime(date,'%Y-%m-%d')+datetime.timedelta(days=7*interval+6))[:10]
    dsl = {'query': {'bool': {'must': [
    {'range': {'paytime': {'gte': begin_date, 'lte': end_date, 'format': 'yyyy-MM-dd', 'time_zone': '+00:00'}}},
    {'terms': {'shop_id': shop_list}}, {'terms': {'order_type': ['vip_scan', 'deal']}},
    {'terms': {'order_status': ['done', 'fulfilled', 'confirmed']}}]}},
     'aggs': {'my_buckets': {'composite': {'size': 9999999, 'sources': [
         {'日期': {'date_histogram': {'field': 'paytime', 'interval': '1w', 'time_zone': '+00:00'}}}]},
                             'aggregations': {'折前GMV': {'sum': {'field': 'skutotal_price'}},
                                              '门店数': {'cardinality': {'field': 'shop_id'}}}}}}
    es = Elasticsearch(['http://172.18.198.27:9200/'], timeout=30)
    res = es.search(index='es_order_table', doc_type='order', body=dsl,
                _source=False, explain=False, track_scores=False, track_total_hits=False, size=0,
                search_type='query_then_fetch')
    if len(res['aggregations']['my_buckets']['buckets'])>0:
        shop_count = res['aggregations']['my_buckets']['buckets'][0]['门店数']['value']
        gmv_count = res['aggregations']['my_buckets']['buckets'][0]['折前GMV']['value']
        order_count = res['aggregations']['my_buckets']['buckets'][0]['doc_count']
    else:
        shop_count = gmv_count = order_count =0
    shop_count_list.append(shop_count)
    gmv_count_list.append(gmv_count)
    order_count_list.append(order_count)
dts[str(interval)+'周后门店数'] = shop_count_list
dts[str(interval)+'周后GMV'] = gmv_count_list
dts[str(interval)+'周后订单数'] = order_count_list
    
    
    
    
    

In [172]:
dts.to_excel(str(interval)+'周门店留存率.xlsx')

In [171]:
dts

,first_date,week_new_shop_list,本周门店数,本周GMV,本周订单数,1周后门店数,1周后GMV,11周后订单数,1周后订单数,2周后门店数,...,5周后订单数,6周后门店数,6周后GMV,6周后订单数,7周后门店数,7周后GMV,7周后订单数,8周后门店数,8周后GMV,8周后订单数
0,2019-01-07,"[10024, 10122, 10158, 10166, 10184, 10188, 101...",119,25492.170003,1794,76,63600.289979,2196,2196,78,...,1492,58,30832.170001,1265,69,34782.319997,1548,60,38577.119991,1511
1,2019-01-14,"[10087, 10349, 10402, 10103, 10022, 10091, 103...",17,1613.040000,65,11,3904.290000,76,76,9,...,16,7,2762.000000,30,3,613.000000,5,6,3514.010000,58
2,2019-01-21,"[10095, 10271, 10451, 10452, 10457, 10464, 104...",18,18379.600013,1197,11,12775.079982,663,663,2,...,403,8,15462.980000,504,7,10626.750000,368,7,12483.590000,330
3,2019-01-28,"[504, 10393]",2,50.900000,3,1,48.000000,2,2,1,...,4,1,231.000000,9,1,93.100000,5,1,418.000000,16
4,2019-02-04,[],0,0.000000,0,0,0.000000,0,0,0,...,0,0,0.000000,0,0,0.000000,0,0,0.000000,0
5,2019-02-11,"[10473, 10118, 10505, 10423, 10507, 10503, 105...",9,1365.860000,59,7,2078.510000,38,38,8,...,130,7,4507.319999,191,5,4597.800000,171,7,5138.000000,216
6,2019-02-18,"[10518, 10525, 10530, 10523, 10526, 10535, 105...",12,1029.810001,74,10,8038.520000,101,101,10,...,81,7,8713.599999,83,7,4103.920002,46,8,19599.580000,151
7,2019-02-25,"[10424, 10555, 10566, 10531, 10541, 10545, 105...",19,11428.749999,362,15,8859.050000,323,323,14,...,490,13,12325.799999,546,11,17673.200000,614,9,17589.500000,641
8,2019-03-04,"[10522, 10592, 10627, 10588, 10605, 10534, 105...",20,14493.640001,554,15,20355.879998,548,548,14,...,940,14,32127.070003,898,12,36941.960002,1013,12,30045.140001,1031
9,2019-03-11,"[10622, 10647, 10653, 10656, 10608, 10662, 106...",18,5720.220000,264,13,9637.829992,334,334,13,...,396,15,11149.019997,443,13,15183.200000,516,13,20129.799995,402
